In [3]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [4]:
# Import base repositories info from grafana
dir_path = os.getcwd() + '/dat/' + 'repos.csv'
df = pd.read_csv(dir_path)

# Parse file function to bring in supporting csvs and ensure the repos are contained in the main file.
# This is necessary since the scripts generating the files may have been run at different times.
def parse_file(file):
    dir_path = os.getcwd() + '/dat/' + file
    data = pd.read_csv(dir_path)
    return data[data["Repository"].isin(df["Repository"])]

display(df.columns)
display(df.info())
df.head()

Index(['Repository', 'owner', 'name_with_owner', 'url', 'homepage_url',
       'description', 'forks', 'is_fork', 'is_mirror', 'is_private',
       'is_archived', 'is_template', 'stars', 'disk_usage',
       'has_issues_enabled', 'has_projects_enabled', 'has_wiki_enabled',
       'merge_commit_allowed', 'rebase_merge_allowed', 'squash_merge_allowed',
       'created_at', 'updated_at', 'pushed_at'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975 entries, 0 to 974
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Repository            975 non-null    object
 1   owner                 975 non-null    object
 2   name_with_owner       975 non-null    object
 3   url                   975 non-null    object
 4   homepage_url          130 non-null    object
 5   description           825 non-null    object
 6   forks                 975 non-null    int64 
 7   is_fork               975 non-null    bool  
 8   is_mirror             975 non-null    bool  
 9   is_private            975 non-null    bool  
 10  is_archived           975 non-null    bool  
 11  is_template           975 non-null    bool  
 12  stars                 975 non-null    int64 
 13  disk_usage            975 non-null    int64 
 14  has_issues_enabled    975 non-null    bool  
 15  has_projects_enabled  975 non-null    bo

None

,Repository,owner,name_with_owner,url,homepage_url,description,forks,is_fork,is_mirror,is_private,is_archived,is_template,stars,disk_usage,has_issues_enabled,has_projects_enabled,has_wiki_enabled,merge_commit_allowed,rebase_merge_allowed,squash_merge_allowed,created_at,updated_at,pushed_at
0,wqbc,bcgov,bcgov/wqbc,https://github.com/bcgov/wqbc,http://bcgov.github.io/wqbc/,An R package for water quality thresholds and ...,9,False,False,False,False,False,14,26448,True,True,True,True,True,True,2014-12-15 19:26:57,2021-01-27 13:45:38,2020-12-16 15:14:54
1,BC-Policy-Framework-For-GitHub,bcgov,bcgov/BC-Policy-Framework-For-GitHub,https://github.com/bcgov/BC-Policy-Framework-F...,NaN,Policy information for BC Government employees...,43,False,False,False,False,False,101,1096,True,True,True,True,True,True,2015-02-26 15:52:57,2021-04-13 00:19:10,2021-02-19 09:56:56
2,bcgroundwater,bcgov,bcgov/bcgroundwater,https://github.com/bcgov/bcgroundwater,NaN,An R package to facilitate analysis and visua...,4,False,False,False,False,False,17,1470,True,True,True,True,True,True,2015-03-13 13:46:25,2021-03-11 07:20:08,2020-12-16 15:28:56
3,groundwater-levels-indicator,bcgov,bcgov/groundwater-levels-indicator,https://github.com/bcgov/groundwater-levels-in...,NaN,R scripts for an indicator on long-term trends...,6,False,False,False,False,False,16,18458,True,True,True,True,True,True,2015-03-13 13:47:17,2021-03-11 06:57:12,2020-12-16 15:16:54
4,ckanext-googleanalytics,bcgov,bcgov/ckanext-googleanalytics,https://github.com/bcgov/ckanext-googleanalytics,NaN,CKAN extension to integrate Google Analytics d...,1,True,False,False,False,False,0,115,False,False,False,True,True,True,2015-04-13 09:16:07,2020-08-09 21:37:35,2020-08-09 21:37:33


In [5]:
def left_merge(main, to_merge):
    return main.merge(to_merge, how="left", on="Repository", suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

In [6]:
# Bring in issues information
issues_df = parse_file('issues.csv')
issues_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23663 entries, 0 to 23678
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Repository    23663 non-null  object
 1   Author        23372 non-null  object
 2   Closed        23663 non-null  bool  
 3   ClosedAt      17071 non-null  object
 4   CreatedAt     23663 non-null  object
 5   LastEditedAt  7967 non-null   object
 6   State         23663 non-null  object
 7   Title         23662 non-null  object
 8   UpdatedAt     23663 non-null  object
 9   Time Open     23663 non-null  object
dtypes: bool(1), object(9)
memory usage: 1.8+ MB


In [7]:
# Function to add repo-mountie open issues count
def add_repomountie_issue_count(titles, issues_df, master):
    df_rm = master
    for title in titles:
        open_issues = issues_df.loc[
            (issues_df['Author'] == 'repo-mountie') &
            (issues_df["Title"] == title) &
            (issues_df["State"] == "OPEN")
        ]
        open_issues_count = open_issues.pivot_table(index=['Repository'], aggfunc='size').to_frame().reset_index().rename(columns={0:title})
        df_rm = left_merge(df_rm, open_issues_count)
    return df_rm


In [8]:
# Add a count column of each repomountie issue type
df_rm = add_repomountie_issue_count([
 "Add project lifecycle badge",
 "Lets use common phrasing",
 "Add missing topics",
], issues_df, df)

# Add total RM issues open
all_open_rm_issues = issues_df.loc[
    (issues_df['Author'] == 'repo-mountie') &
    (issues_df["State"] == "OPEN")
]
total_open_rm_issues = all_open_rm_issues.pivot_table(index=['Repository'], aggfunc='size').to_frame().reset_index().rename(columns={0:"Open RM issues"})
df_rm = left_merge(df_rm, total_open_rm_issues)

# Give clearer names to the new columns and change NaN to 0
df_rm.rename(columns = {
    "Add project lifecycle badge": "Open lifecycle badge issues",
    "Lets use common phrasing": "Open common phrasing issues",
    "Add missing topics": "Open missing topics issues"
}, inplace=True)
df_rm.fillna({'Open missing topics issues':0, 'Open common phrasing issues':0, "Open lifecycle badge issues": 0, "Open RM issues": 0}, inplace=True)
df_rm.head()

,Repository,owner,name_with_owner,url,homepage_url,description,forks,is_fork,is_mirror,is_private,is_archived,is_template,stars,disk_usage,has_issues_enabled,has_projects_enabled,has_wiki_enabled,merge_commit_allowed,rebase_merge_allowed,squash_merge_allowed,created_at,updated_at,pushed_at,Open lifecycle badge issues,Open common phrasing issues,Open missing topics issues,Open RM issues
0,wqbc,bcgov,bcgov/wqbc,https://github.com/bcgov/wqbc,http://bcgov.github.io/wqbc/,An R package for water quality thresholds and ...,9,False,False,False,False,False,14,26448,True,True,True,True,True,True,2014-12-15 19:26:57,2021-01-27 13:45:38,2020-12-16 15:14:54,0.0,0.0,0.0,0.0
1,BC-Policy-Framework-For-GitHub,bcgov,bcgov/BC-Policy-Framework-For-GitHub,https://github.com/bcgov/BC-Policy-Framework-F...,NaN,Policy information for BC Government employees...,43,False,False,False,False,False,101,1096,True,True,True,True,True,True,2015-02-26 15:52:57,2021-04-13 00:19:10,2021-02-19 09:56:56,1.0,0.0,0.0,1.0
2,bcgroundwater,bcgov,bcgov/bcgroundwater,https://github.com/bcgov/bcgroundwater,NaN,An R package to facilitate analysis and visua...,4,False,False,False,False,False,17,1470,True,True,True,True,True,True,2015-03-13 13:46:25,2021-03-11 07:20:08,2020-12-16 15:28:56,0.0,0.0,0.0,0.0
3,groundwater-levels-indicator,bcgov,bcgov/groundwater-levels-indicator,https://github.com/bcgov/groundwater-levels-in...,NaN,R scripts for an indicator on long-term trends...,6,False,False,False,False,False,16,18458,True,True,True,True,True,True,2015-03-13 13:47:17,2021-03-11 06:57:12,2020-12-16 15:16:54,0.0,0.0,0.0,0.0
4,ckanext-googleanalytics,bcgov,bcgov/ckanext-googleanalytics,https://github.com/bcgov/ckanext-googleanalytics,NaN,CKAN extension to integrate Google Analytics d...,1,True,False,False,False,False,0,115,False,False,False,True,True,True,2015-04-13 09:16:07,2020-08-09 21:37:35,2020-08-09 21:37:33,0.0,0.0,0.0,0.0


In [9]:
topics_df = parse_file('repo-topics.csv')

topics_count = topics_df.groupby('Repository').agg("count").reset_index()
df_rm = left_merge(df_rm, topics_count[["Repository", "Ministry Code"]])
df_rm.rename(columns = {
    "Ministry Code": "Ministry Codes Count"
}, inplace=True)
df_rm.head()

,Repository,owner,name_with_owner,url,homepage_url,description,forks,is_fork,is_mirror,is_private,is_archived,is_template,stars,disk_usage,has_issues_enabled,has_projects_enabled,has_wiki_enabled,merge_commit_allowed,rebase_merge_allowed,squash_merge_allowed,created_at,updated_at,pushed_at,Open lifecycle badge issues,Open common phrasing issues,Open missing topics issues,Open RM issues,Ministry Codes Count
0,wqbc,bcgov,bcgov/wqbc,https://github.com/bcgov/wqbc,http://bcgov.github.io/wqbc/,An R package for water quality thresholds and ...,9,False,False,False,False,False,14,26448,True,True,True,True,True,True,2014-12-15 19:26:57,2021-01-27 13:45:38,2020-12-16 15:14:54,0.0,0.0,0.0,0.0,1.0
1,BC-Policy-Framework-For-GitHub,bcgov,bcgov/BC-Policy-Framework-For-GitHub,https://github.com/bcgov/BC-Policy-Framework-F...,NaN,Policy information for BC Government employees...,43,False,False,False,False,False,101,1096,True,True,True,True,True,True,2015-02-26 15:52:57,2021-04-13 00:19:10,2021-02-19 09:56:56,1.0,0.0,0.0,1.0,1.0
2,bcgroundwater,bcgov,bcgov/bcgroundwater,https://github.com/bcgov/bcgroundwater,NaN,An R package to facilitate analysis and visua...,4,False,False,False,False,False,17,1470,True,True,True,True,True,True,2015-03-13 13:46:25,2021-03-11 07:20:08,2020-12-16 15:28:56,0.0,0.0,0.0,0.0,1.0
3,groundwater-levels-indicator,bcgov,bcgov/groundwater-levels-indicator,https://github.com/bcgov/groundwater-levels-in...,NaN,R scripts for an indicator on long-term trends...,6,False,False,False,False,False,16,18458,True,True,True,True,True,True,2015-03-13 13:47:17,2021-03-11 06:57:12,2020-12-16 15:16:54,0.0,0.0,0.0,0.0,1.0
4,ckanext-googleanalytics,bcgov,bcgov/ckanext-googleanalytics,https://github.com/bcgov/ckanext-googleanalytics,NaN,CKAN extension to integrate Google Analytics d...,1,True,False,False,False,False,0,115,False,False,False,True,True,True,2015-04-13 09:16:07,2020-08-09 21:37:35,2020-08-09 21:37:33,0.0,0.0,0.0,0.0,1.0


In [10]:
repolint_df = parse_file('repolint-results.csv')

In [11]:
cols_to_drop = [
    'javascript-package-metadata-exists', 
    'ruby-package-metadata-exists',
    'java-package-metadata-exists', 
    'python-package-metadata-exists',
    'objective-c-package-metadata-exists', 
    'swift-package-metadata-exists',
    'erlang-package-metadata-exists', 
    'elixir-package-metadata-exists',
    'license-detectable-by-licensee',
    'notice-file-exists'
]

repolint_df.drop(cols_to_drop, axis=1, inplace=True)
repolint_df.replace(inplace=True, to_replace="PASSED", value=True)
repolint_df.replace(inplace=True, to_replace="NOT_PASSED*", value=False, regex=True)
repolint_df.head()

df_rm.shape


(975, 28)

In [12]:
df_rm = left_merge(df_rm, repolint_df)

In [13]:
df_repo_1 = parse_file("repository-details-1.csv")
df_repo_2 = parse_file("repository-details-2.csv")
df_rm = left_merge(df_rm, df_repo_1)
df_rm = left_merge(df_rm, df_repo_2)
df_rm.head()

,Repository,owner,name_with_owner,url,homepage_url,description,forks,is_fork,is_mirror,is_private,is_archived,is_template,stars,disk_usage,has_issues_enabled,has_projects_enabled,has_wiki_enabled,merge_commit_allowed,rebase_merge_allowed,squash_merge_allowed,created_at,updated_at,pushed_at,Open lifecycle badge issues,Open common phrasing issues,Open missing topics issues,Open RM issues,Ministry Codes Count,license-file-exists,readme-file-exists,contributing-file-exists,code-of-conduct-file-exists,changelog-file-exists,security-file-exists,support-file-exists,readme-references-license,binaries-not-present,test-directory-exists,integrates-with-ci,code-of-conduct-file-contains-email,source-license-headers-exist,github-issue-template-exists,github-pull-request-template-exists,PackageCount,ProjectCount,ReleaseCount,SubmoduleCount,DeployKeyCount,TopicCount,License,CodeOfConduct,DaysOpen,IssueCount,PrCount,CommitCount,AverageIssueCountPerDay,AveragePrCountPerDay,AverageCommitCountPerDay,DefaultBranchName,Languages
0,wqbc,bcgov,bcgov/wqbc,https://github.com/bcgov/wqbc,http://bcgov.github.io/wqbc/,An R package for water quality thresholds and ...,9,False,False,False,False,False,14,26448,True,True,True,True,True,True,2014-12-15 19:26:57,2021-01-27 13:45:38,2020-12-16 15:14:54,0.0,0.0,0.0,0.0,1.0,True,True,True,True,False,False,False,True,True,True,True,True,False,False,False,0.0,0.0,9.0,0.0,0.0,5.0,Apache License 2.0,Contributor Covenant,2319.0,104.0,56.0,778.0,0.04,0.02,0.34,master,R_TeX_Rebol
1,BC-Policy-Framework-For-GitHub,bcgov,bcgov/BC-Policy-Framework-For-GitHub,https://github.com/bcgov/BC-Policy-Framework-F...,NaN,Policy information for BC Government employees...,43,False,False,False,False,False,101,1096,True,True,True,True,True,True,2015-02-26 15:52:57,2021-04-13 00:19:10,2021-02-19 09:56:56,1.0,0.0,0.0,1.0,1.0,True,True,True,True,False,False,False,True,True,False,False,True,False,False,False,0.0,0.0,1.0,0.0,0.0,8.0,Other,Contributor Covenant,2246.0,20.0,43.0,186.0,0.01,0.02,0.08,master,NaN
2,bcgroundwater,bcgov,bcgov/bcgroundwater,https://github.com/bcgov/bcgroundwater,NaN,An R package to facilitate analysis and visua...,4,False,False,False,False,False,17,1470,True,True,True,True,True,True,2015-03-13 13:46:25,2021-03-11 07:20:08,2020-12-16 15:28:56,0.0,0.0,0.0,0.0,1.0,True,True,True,True,False,False,False,True,True,True,True,True,False,False,False,0.0,0.0,2.0,0.0,0.0,4.0,Apache License 2.0,Contributor Covenant,2231.0,19.0,14.0,256.0,0.01,0.01,0.11,master,R_Makefile
3,groundwater-levels-indicator,bcgov,bcgov/groundwater-levels-indicator,https://github.com/bcgov/groundwater-levels-in...,NaN,R scripts for an indicator on long-term trends...,6,False,False,False,False,False,16,18458,True,True,True,True,True,True,2015-03-13 13:47:17,2021-03-11 06:57:12,2020-12-16 15:16:54,0.0,0.0,0.0,0.0,1.0,True,True,True,True,False,False,False,True,True,False,False,True,False,False,False,0.0,0.0,0.0,0.0,0.0,5.0,Apache License 2.0,Contributor Covenant,2231.0,12.0,5.0,218.0,0.01,0.00,0.10,master,R_CSS_TeX
4,ckanext-googleanalytics,bcgov,bcgov/ckanext-googleanalytics,https://github.com/bcgov/ckanext-googleanalytics,NaN,CKAN extension to integrate Google Analytics d...,1,True,False,False,False,False,0,115,False,False,False,True,True,True,2015-04-13 09:16:07,2020-08-09 21:37:35,2020-08-09 21:37:33,0.0,0.0,0.0,0.0,1.0,False,True,False,False,False,False,False,True,True,True,False,True,False,False,False,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,2201.0,0.0,6.0,105.0,0.00,0.00,0.05,master,Python_JavaScript_HTML


In [20]:
df_forks = parse_file('repository-fork-pr.csv')
df_pr = parse_file('repository-pr.csv')

df_pr.head()

,Repository,Fork PullRequest Count,Review Count
0,wqbc,0,16
1,BC-Policy-Framework-For-GitHub,0,11
2,bcgroundwater,0,4
3,groundwater-levels-indicator,0,1
4,ckanext-googleanalytics,0,0


In [21]:
df_rm = left_merge(df_rm, df_forks)
df_rm = left_merge(df_rm, df_pr)

In [22]:
df_rm.info()
# df_rm.to_csv(os.getcwd() + '/dat/master.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 62 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Repository                           975 non-null    object 
 1   owner                                975 non-null    object 
 2   name_with_owner                      975 non-null    object 
 3   url                                  975 non-null    object 
 4   homepage_url                         130 non-null    object 
 5   description                          825 non-null    object 
 6   forks                                975 non-null    int64  
 7   is_fork                              975 non-null    bool   
 8   is_mirror                            975 non-null    bool   
 9   is_private                           975 non-null    bool   
 10  is_archived                          975 non-null    bool   
 11  is_template                     